# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.17,83,95,17.03,SH,1721759316
1,1,samfya,-11.3649,29.5565,20.97,26,0,4.94,ZM,1721759317
2,2,san antonio de pale,-1.4014,5.6325,22.99,85,44,4.61,GQ,1721759319
3,3,la ronge,55.1001,-105.2842,21.14,56,40,7.72,CA,1721759320
4,4,papatowai,-46.5619,169.4708,8.38,72,62,4.44,NZ,1721759321


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",  
)

# Display the map
city_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,villa de cos,23.2833,-102.3500,26.87,32,0,2.86,MX,1721759435
108,108,n'zeto,-7.2312,12.8666,21.31,90,0,3.67,AO,1721759452
132,132,victoria falls,-17.9235,25.8430,22.06,26,0,2.06,ZW,1721759482
139,139,kuusamo,65.9667,29.1833,24.02,41,0,2.06,FI,1721759492
181,181,bilecik,40.0000,30.1667,24.36,70,0,2.67,TR,1721759547
218,218,kittila,67.6647,24.8936,23.08,49,0,3.09,FI,1721759592
225,225,hammerfest,70.6634,23.6821,24.25,53,0,2.06,NO,1721759600
226,226,ruidoso,33.3317,-105.6730,23.36,46,0,2.57,US,1721759601
227,227,luanda,-8.8368,13.2343,23.00,73,0,3.09,AO,1721759492
276,276,damavand,35.7178,52.0650,23.71,31,0,0.63,IR,1721759662


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/bs/kbsf579n5k30mfl6t819tlhc0000gn/T/ipykernel_15707/56385462.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
95,villa de cos,MX,23.2833,-102.3500,32,
108,n'zeto,AO,-7.2312,12.8666,90,
132,victoria falls,ZW,-17.9235,25.8430,26,
139,kuusamo,FI,65.9667,29.1833,41,
181,bilecik,TR,40.0000,30.1667,70,
218,kittila,FI,67.6647,24.8936,49,
225,hammerfest,NO,70.6634,23.6821,53,
226,ruidoso,US,33.3317,-105.6730,46,
227,luanda,AO,-8.8368,13.2343,73,
276,damavand,IR,35.7178,52.0650,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
villa de cos - nearest hotel: No hotel found
n'zeto - nearest hotel: Longe LUVUVAMO
victoria falls - nearest hotel: Shearwater Explorers Village
kuusamo - nearest hotel: Holiday Club Kuusamon Tropiikki
bilecik - nearest hotel: Namlı Osmanlı Otel
kittila - nearest hotel: Hotelli Kittilä
hammerfest - nearest hotel: Thon Hotel Hammerfest
ruidoso - nearest hotel: Hotel Ruidoso
luanda - nearest hotel: Hotel Forum
damavand - nearest hotel: Avishan Hotel
vadso - nearest hotel: Scandic Vadsø
la passe - nearest hotel: L'Estuaire
astrakhan - nearest hotel: Omega
nova vicosa - nearest hotel: Pousada Pontal da Barra
inhambane - nearest hotel: Hotel de Inhambane
prado - nearest hotel: Pousada Recanto do Prado
vernon - nearest hotel: Okanagan Royal Park Inn
oulunsalo - nearest hotel: Finlandia Hotel Airport Oulu
montrose - nearest hotel: No hotel found
alamosa - nearest hotel: 40 Winks Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
95,villa de cos,MX,23.2833,-102.3500,32,No hotel found
108,n'zeto,AO,-7.2312,12.8666,90,Longe LUVUVAMO
132,victoria falls,ZW,-17.9235,25.8430,26,Shearwater Explorers Village
139,kuusamo,FI,65.9667,29.1833,41,Holiday Club Kuusamon Tropiikki
181,bilecik,TR,40.0000,30.1667,70,Namlı Osmanlı Otel
218,kittila,FI,67.6647,24.8936,49,Hotelli Kittilä
225,hammerfest,NO,70.6634,23.6821,53,Thon Hotel Hammerfest
226,ruidoso,US,33.3317,-105.6730,46,Hotel Ruidoso
227,luanda,AO,-8.8368,13.2343,73,Hotel Forum
276,damavand,IR,35.7178,52.0650,31,Avishan Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)